# AIM Hackathon: Sample code
19.10.2024

Note: This is just some code to help you getting started, but in no way mandatory to use. Feel free to use any other tools, libraries, approaches, etc.

In [18]:
import os
import requests
import PyPDF2
import pandas as pd
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.documents import Document

# load openai key
if not load_dotenv():
    raise Exception('Error loading .env file. Make sure to place a valid OPEN_AI_KEY in the .env file.')

In [19]:
# TODO set global variables
REPORTS_SAVE_PATH = 'data/sample_reports'
DB_PATH = "data/db/sample.db"

# See https://openai.com/api/pricing/
MODEL = "gpt-4o"

## Data

In [20]:
df = pd.read_json('data/reports.json')
df

,company_name,year,dataset,pdf_url
0,Walmart,2023,handcrafted,https://corporate.walmart.com/content/dam/corp...
1,Walmart,2021,handcrafted,https://corporate.walmart.com/content/dam/corp...
2,Walmart,2019,handcrafted,https://corporate.walmart.com/content/dam/corp...
3,Amazon,2023,handcrafted,https://sustainability.aboutamazon.com/content...
4,Amazon,2021,handcrafted,https://sustainability.aboutamazon.com/content...
...,...,...,...,...
141,tarkett,2020,scraped,https://www.tarkett.com/sites/default/files/20...
142,trivium-packaging,2021,scraped,https://www.triviumpackaging.com/media/13fl4q3...
143,trivium-packaging,2020,scraped,https://triviumpackaging.com/sustainability/re...
144,trust,2023,scraped,https://dezlwerqy1h00.cloudfront.net/images/co...


## Download some reports

In [21]:
# EXAMPLE: select Apple reports
df_sample = df[df['company_name'] == 'Apple']

In [22]:
# download Apple reports to save_dir
def download_files(df: pd.DataFrame, save_dir: str):
    os.makedirs(save_dir, exist_ok=True)
    for url in df['pdf_url']:
        pdf_filename = os.path.basename(url)
        response = requests.get(url)
        with open(os.path.join(save_dir, pdf_filename), 'wb') as file:
            file.write(response.content)
    print(f"Success.")

In [ ]:
download_files(df_sample, REPORTS_SAVE_PATH)

## Load PDFs as documents

In [8]:
def get_documents_from_path(files_path: str) -> [Document]:
    documents = []
    
    for file in os.listdir(files_path):
        _, file_extension = os.path.splitext(file)
        text = ""
        
        if file_extension == ".pdf":
            with open(os.path.join(files_path, file), 'rb') as f:
                reader = PyPDF2.PdfReader(f, strict=False)
                for page in reader.pages:
                    text += page.extract_text() + "\n"
                
            if text:
                documents.append(Document(page_content=text, metadata={"source": file}))
            else:
                print(f"WARNING: No text extracted from {file}")
        else:
            # TODO: can add support for other file types here
            print(f" WARNING: Unsupported file extension: {file_extension}")
    
    return documents

In [9]:
documents = get_documents_from_path(REPORTS_SAVE_PATH)

## Create simple vector database

In [17]:
# TODO Create database
retriever = None

## Create simple RAG

In [14]:
# Load the LLM
llm = ChatOpenAI(model_name=MODEL, temperature=0)  # for deterministic outputs

# TODO Load retriever, question answer pipeline, etc.

In [15]:
def ask_question():
    pass

In [16]:
response = ask_question("When does Apple try to achieve carbon neutrality?")

Question: When does Apply try to achieve carbon neutrality?
Answer: Apple aims to achieve carbon neutrality for its entire carbon footprint, including products, by 2030.
